# Testes Wellzesta

## Instalação do Selenium

In [73]:
%pip install selenium webdriver_manager beautifulsoup4
%pip install openai langchain pandas ipywidgets

from IPython.display import clear_output
clear_output()

print("Done!")

Done!


## Source Code

In [74]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
from collections import defaultdict
from xml.sax.saxutils import quoteattr

import pandas as pd
import ipywidgets as widgets
from IPython.display import display

import json
import time


prefs = {"profile.default_content_setting_values.notifications": 1}
chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.quit()


def new_window(url, wait=0):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    time.sleep(wait)
    return driver


def scroll(window, tagname, text, wait=0):
    find_element(window, tagname, text).location_once_scrolled_into_view
    time.sleep(wait)


def scroll_and_click(window, tagname, text, wait=0):
    e = find_element(window=window, tagname=tagname, text=text)
    e.location_once_scrolled_into_view
    e.click()
    time.sleep(wait)


def wait(seconds):
    time.sleep(seconds)


def get_attribute(window, mode, key, attribute):
    return window.find_element(mode, key).get_attribute(attribute)


def get_communitylogo_url(window, text=None, clazz=None):
    img = find_element(window, tagname="img", text=text, clazz=clazz)
    return img.get_attribute("src")


def find_element(window, text=None, clazz=None, placeholder=None, id=None, tagname="*", retry=5, delay=3): 
    for i in range(retry):
        try:
            elements = window.find_elements(By.TAG_NAME, tagname)
            for element in elements:
                if id!=None and element.get_attribute("id")==id: return element
                if text!=None and element.text==text: return element
                if clazz!=None and element.get_attribute("class")==clazz: return element
                if placeholder!=None and element.get_attribute("placeholder")==placeholder: return element
        except: pass
        time.sleep(delay)


def type(window, value, id=None, placeholder=None, wait=0):
    element = find_element(window, tagname="input", placeholder=placeholder, id=id)
    element.send_keys(value)
    time.sleep(wait)


def click(window, text=None, tagname="button", wait=3):
    find_element(window, tagname=tagname, text=text).click()


def match(value1, value2, tip="test:"):
    print(tip)
    print("\texpected: ", value1)
    print("\tcurrent: ", value2)

    try:
        assert value1 == value2, f"{value1} != {value2}"
        print("\tsuccess!")
        
    except AssertionError as e:
        print("\tfailed!")
        raise e


def active_login_superadmin(email="anelisedaux@gmail.com", password="12345678", wait=3):
    active = new_window("https://active.stg.wellzesta.com/wz-login")
    type(active, email, placeholder='Type your e-mail')
    type(active, password, placeholder='Type your password')
    click(active, "Sign In")
    time.sleep(wait)
    return active


def active_get_communitylogo_url(window):
    click(window, tagname="a", text="Organization Details")
    return get_communitylogo_url(window, clazz="Image-sc-nke03c-0 sxPXz")

print("Done!")

Done!


## Reusable Functions

### Activate

In [75]:

def active_login_superadmin(email="anelisedaux@gmail.com", password="12345678", wait=3):
    active = new_window("https://active.stg.wellzesta.com/wz-login")
    type(active, email, placeholder='Type your e-mail')
    type(active, password, placeholder='Type your password')
    click(active, "Sign In")
    time.sleep(wait)
    return active


def active_get_communitylogo_url(window):
    return get_communitylogo_url(window, clazz="Image-sc-nke03c-0 sxPXz")


### Life

In [76]:

def life_login_resident(email="yoda@gmail.com", pin="1234", wait=2):
    life = new_window("https://life.stg.wellzesta.com/login")

    type(life, email, placeholder='Type your email')
    click(life, "Sign-in using PIN")

    type(life, pin, placeholder="PIN")
    click(life, "Login")

    time.sleep(wait)
    return life


def life_get_communitylogo_url(window):
    return get_communitylogo_url(window, clazz="w-8 h-8 rounded-md")


## Test: CommunityLogo

In [77]:

life = life_login_resident()

life_logo_url = life_get_communitylogo_url(life)
life.quit()

active = active_login_superadmin()
scroll_and_click(active, tagname="b", text="Life Org - Florianopolis, AK")
click(active, tagname="a", text="Organization Details")

active_logo_url = active_get_communitylogo_url(active)
active.quit()

try:
    match(active_logo_url, life_logo_url, "Teste 01: Verifica Logo da Organização")
except:
    pass


Teste 01: Verifica Logo da Organização
	expected:  https://cdn.filestackcontent.com/llb0AQv3TdS63QPEajWn
	current:  https://life.stg.wellzesta.com/static/images/avafield.png
	failed!
